In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

import tams
from tams.idealized import Blob, Field

In [ ]:
Blob().get_tendency()

In [ ]:
Blob().polygon

In [ ]:
Blob(c=(1, 2)).split()

In [ ]:
Field(Blob(c=(1, 2), a=3, b=1, theta=20).split(3)).to_geopandas().plot(fc="none")

In [ ]:
b1 = Blob(a=3, b=1, theta=30)
b2 = Blob(a=4, b=2)
display(b1.ring)
display(b2.ring)
b3 = b1.merge(b2)
display(b3.ring)

In [ ]:
Field()

In [ ]:
%%time

b = Blob(c=(0, 0), a=5, b=3)

ctt = Field(b).to_xarray()

(ce,), _ = tams.identify(ctt)

fig, ax = plt.subplots(figsize=(10, 6))

ctt.plot(ax=ax)
gpd.GeoSeries(b.ring).plot(ec="magenta", ax=ax)

ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none", zorder=10)

ax.set_aspect("equal")

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True, layout="constrained")

ctt.sel(lat=0, method="nearest").plot(ax=ax1)
ctt.sel(lon=0, method="nearest").plot(ax=ax2)

ax2.set_ylabel("")
for ax in fig.get_axes():
    ax.axhline(235, c="0.35", ls=":")